# Text Preprocessing

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import numpy as np
from gensim.models import Word2Vec
import re

class TextPreprocessor:
    def __init__(self):
        pass
        
    def clean_text(self, text):
        if isinstance(text, float):
            return ""
        text = text.lower()
        # https://medium.com/@siddharthgov01/regular-expressions-from-a-za-z-88cf9cf0abac
        text = re.sub(r'[^a-zA-Z\s]', '', text)
        return text
    

    # https://pages.github.rpi.edu/kuruzj/website_introml_rpi/notebooks/08-intro-nlp/03-scikit-learn-text.html#bag-of-words-using-scikit-learn
    def get_bow_features(self, texts, max_features=5000):
        vectorizer = CountVectorizer(max_features=max_features)
        return vectorizer.fit_transform(texts)

    # https://pages.github.rpi.edu/kuruzj/website_introml_rpi/notebooks/08-intro-nlp/03-scikit-learn-text.html#tf-idf-encoding
    def get_tfidf_features(self, texts, max_features=5000):
        vectorizer = TfidfVectorizer(max_features=max_features)
        return vectorizer.fit_transform(texts)
 
    # https://radimrehurek.com/gensim/models/word2vec.html
    def get_word2vec_features(self, texts, vector_size=100, window=5, min_count=1, workers=4):
        processed_texts = []
        for text in texts:
            if isinstance(text, str):
                cleaned = self.clean_text(text)
                tokens = cleaned.split()
                processed_texts.append(tokens)

        model = Word2Vec(
            sentences=processed_texts,
            vector_size=vector_size,
            window=window,
            min_count=min_count,
            workers=workers
        )
        
        return model

    def get_text_vector(self, text, word2vec_model):
        tokens = self.clean_text(text).split()
        vectors = []
        for token in tokens:
            if token in word2vec_model.wv:
                vectors.append(word2vec_model.wv[token])
                
        if vectors:
            return np.mean(vectors, axis=0)
        return np.zeros(word2vec_model.vector_size)

import pandas as pd
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')




# Linear Regression with TI-IDF Word Processor

In [2]:

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

preprocessor = TextPreprocessor()
train_texts = train_df['text'].apply(preprocessor.clean_text)
train_labels = train_df['label']

X_train, X_test, y_train, y_test = train_test_split(train_texts, train_labels, test_size=0.2, random_state=42, stratify=train_labels)

X_train_tfidf = preprocessor.get_tfidf_features(X_train)
X_test_tfidf = preprocessor.get_tfidf_features(X_test)

lr_classifier = LogisticRegression(max_iter=1000, C=5.0, penalty='l2', random_state=42 )


lr_classifier.fit(X_train_tfidf, y_train)


y_pred = lr_classifier.predict(X_test_tfidf)


accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")


print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 50.91%

Classification Report:
              precision    recall  f1-score   support

           0       0.63      0.04      0.08      2077
           1       0.51      0.98      0.67      2083

    accuracy                           0.51      4160
   macro avg       0.57      0.51      0.37      4160
weighted avg       0.57      0.51      0.37      4160



# Linear Regression with BOW

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report


preprocessor = TextPreprocessor()
train_texts = train_df['text'].apply(preprocessor.clean_text)
train_labels = train_df['label']

X_train, X_test, y_train, y_test = train_test_split(train_texts, train_labels, test_size=0.2, random_state=42, stratify=train_labels)

X_train_tfidf = preprocessor.get_bow_features(X_train)
X_test_tfidf = preprocessor.get_bow_features (X_test)

lr_classifier = LogisticRegression(max_iter=1000, C=5.0, penalty='l2', random_state=42 )



lr_classifier.fit(X_train_tfidf, y_train)


y_pred = lr_classifier.predict(X_test_tfidf)


accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")


print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 59.88%

Classification Report:
              precision    recall  f1-score   support

           0       0.57      0.79      0.66      2077
           1       0.66      0.40      0.50      2083

    accuracy                           0.60      4160
   macro avg       0.62      0.60      0.58      4160
weighted avg       0.62      0.60      0.58      4160



# Linear Regression with Word2vec Embedding

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report


preprocessor = TextPreprocessor()
train_texts = train_df['text'].apply(preprocessor.clean_text)
train_labels = train_df['label']

X_train, X_test, y_train, y_test = train_test_split(train_texts, train_labels, test_size=0.2, random_state=42, stratify=train_labels)

X_train_text_vectors = []
X_test_text_vectors = []

word2vec_model = preprocessor.get_word2vec_features(X_train)

for text in X_train:
    text_vector = preprocessor.get_text_vector(text, word2vec_model)
    X_train_text_vectors.append(text_vector)
    
for text in X_test:
    text_vector = preprocessor.get_text_vector(text, word2vec_model)
    X_test_text_vectors.append(text_vector)

X_train_text_vectors = pd.DataFrame(X_train_text_vectors)
X_test_text_vectors = pd.DataFrame(X_test_text_vectors)

lr_classifier = LogisticRegression(max_iter=1000, C=5.0, penalty='l2', solver="saga", random_state=42)

lr_classifier.fit(X_train_text_vectors, y_train)

y_train_pred = lr_classifier.predict(X_train_text_vectors)
y_test_pred = lr_classifier.predict(X_test_text_vectors)

train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"Train Accuracy: {train_accuracy * 100:.2f}%")
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Train Accuracy: 89.39%
Test Accuracy: 88.32%

Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.90      0.89      2077
           1       0.89      0.87      0.88      2083

    accuracy                           0.88      4160
   macro avg       0.88      0.88      0.88      4160
weighted avg       0.88      0.88      0.88      4160



/home/kevin/.local/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


# MLP Model with BOW

In [3]:
import keras
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.layers import Dense, Dropout
from keras.models import Sequential
from sklearn.metrics import accuracy_score, classification_report

# Source https://www.tensorflow.org/text/tutorials/text_classification_rnn

preprocessor = TextPreprocessor()

train_df = pd.read_csv('train.csv')
train_texts = train_df['text'].apply(preprocessor.clean_text)
train_labels = train_df['label']


X_train, X_test, y_train, y_test = train_test_split(train_texts, train_labels, test_size=0.2, random_state=42, stratify=train_labels)

X_train_bow = preprocessor.get_bow_features(X_train).toarray()
X_test_bow = preprocessor.get_bow_features (X_test).toarray()

num_classes = len(set(train_labels))
y_train_encoded = keras.utils.to_categorical(y_train, num_classes)
y_test_encoded = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train_bow.shape[1],)))
model.add(Dropout(0.3))  
model.add(Dense(64, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer= "Adam", metrics=['accuracy'])


model.fit(
    X_train_bow,
    y_train_encoded,
    batch_size=32,
    epochs=10,
    validation_split=0.2
)


y_pred_prob = model.predict(X_test_bow)  
y_pred = np.argmax(y_pred_prob, axis=1)   
y_test_labels = np.argmax(y_test_encoded, axis=1) 

print("Accuracy:",  accuracy_score(y_test_labels, y_pred))
print(classification_report(y_test_labels, y_pred))


/home/kevin/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-11-28 19:06:29.053039: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-28 19:06:31.162070: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 532480000 exceeds 10% of free system memory.


Epoch 1/10
414/416 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.8510 - loss: 0.3732

2024-11-28 19:06:55.443060: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 133120000 exceeds 10% of free system memory.


416/416 ━━━━━━━━━━━━━━━━━━━━ 16s 28ms/step - accuracy: 0.8513 - loss: 0.3726 - val_accuracy: 0.9396 - val_loss: 0.1747
Epoch 2/10
416/416 ━━━━━━━━━━━━━━━━━━━━ 10s 23ms/step - accuracy: 0.9617 - loss: 0.1160 - val_accuracy: 0.9384 - val_loss: 0.1863
Epoch 3/10
416/416 ━━━━━━━━━━━━━━━━━━━━ 14s 33ms/step - accuracy: 0.9727 - loss: 0.0827 - val_accuracy: 0.9489 - val_loss: 0.1670
Epoch 4/10
416/416 ━━━━━━━━━━━━━━━━━━━━ 9s 21ms/step - accuracy: 0.9913 - loss: 0.0289 - val_accuracy: 0.9504 - val_loss: 0.1974
Epoch 5/10
416/416 ━━━━━━━━━━━━━━━━━━━━ 9s 21ms/step - accuracy: 0.9919 - loss: 0.0299 - val_accuracy: 0.9453 - val_loss: 0.2170
Epoch 6/10
416/416 ━━━━━━━━━━━━━━━━━━━━ 8s 20ms/step - accuracy: 0.9891 - loss: 0.0393 - val_accuracy: 0.9483 - val_loss: 0.2772
Epoch 7/10
416/416 ━━━━━━━━━━━━━━━━━━━━ 8s 20ms/step - accuracy: 0.9907 - loss: 0.0306 - val_accuracy: 0.9564 - val_loss: 0.2610
Epoch 8/10
416/416 ━━━━━━━━━━━━━━━━━━━━ 9s 20ms/step - accuracy: 0.9972 - loss: 0.0108 - val_accuracy: 0.

2024-11-28 19:08:20.524111: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 166400000 exceeds 10% of free system memory.


130/130 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
Accuracy: 0.5014423076923077
              precision    recall  f1-score   support

           0       0.51      0.06      0.11      2077
           1       0.50      0.94      0.65      2083

    accuracy                           0.50      4160
   macro avg       0.50      0.50      0.38      4160
weighted avg       0.50      0.50      0.38      4160



# BiLSTM Model with TDIDF 

In [ ]:
import keras
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.layers import Dense, Dropout
from keras.models import Sequential
from sklearn.metrics import accuracy_score, classification_report

# Source https://www.tensorflow.org/text/tutorials/text_classification_rnn

preprocessor = TextPreprocessor()

train_df = pd.read_csv('train.csv')
train_texts = train_df['text'].apply(preprocessor.clean_text)
train_labels = train_df['label']


X_train, X_test, y_train, y_test = train_test_split(
    train_texts, train_labels, test_size=0.2, random_state=42, stratify=train_labels
)


X_train_tfidf = preprocessor.get_tfidf_features(X_train).toarray()
X_test_tfidf = preprocessor.get_tfidf_features(X_test).toarray()

num_classes = len(set(train_labels))
y_train_encoded = keras.utils.to_categorical(y_train, num_classes)
y_test_encoded = keras.utils.to_categorical(y_test, num_classes)


model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train_tfidf.shape[1],)))
model.add(Dropout(0.3))  
model.add(Dense(64, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


model.fit(
    X_train_tfidf,
    y_train_encoded,
    batch_size=32,
    epochs=10,
    validation_split=0.2
)


y_pred_prob = model.predict(X_test_tfidf)  
y_pred = np.argmax(y_pred_prob, axis=1)   
y_test_labels = np.argmax(y_test_encoded, axis=1) 

print("Accuracy:",  accuracy_score(y_test_labels, y_pred))
print(classification_report(y_test_labels, y_pred))


2024-11-28 15:40:16.984818: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732837217.632622   21194 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732837217.914678   21194 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-28 15:40:18.668932: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/kevin/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/

Epoch 1/10


2024-11-28 15:41:10.198319: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 266240000 exceeds 10% of free system memory.


415/416 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.8262 - loss: 0.3590

2024-11-28 15:41:28.388313: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 66560000 exceeds 10% of free system memory.


416/416 ━━━━━━━━━━━━━━━━━━━━ 14s 29ms/step - accuracy: 0.8266 - loss: 0.3584 - val_accuracy: 0.9459 - val_loss: 0.1389
Epoch 2/10
416/416 ━━━━━━━━━━━━━━━━━━━━ 7s 16ms/step - accuracy: 0.9779 - loss: 0.0634 - val_accuracy: 0.9453 - val_loss: 0.1533
Epoch 3/10
416/416 ━━━━━━━━━━━━━━━━━━━━ 7s 17ms/step - accuracy: 0.9943 - loss: 0.0237 - val_accuracy: 0.9441 - val_loss: 0.1820
Epoch 4/10
416/416 ━━━━━━━━━━━━━━━━━━━━ 7s 18ms/step - accuracy: 0.9972 - loss: 0.0111 - val_accuracy: 0.9417 - val_loss: 0.2199
Epoch 5/10
416/416 ━━━━━━━━━━━━━━━━━━━━ 8s 20ms/step - accuracy: 0.9996 - loss: 0.0037 - val_accuracy: 0.9447 - val_loss: 0.2642
Epoch 6/10
416/416 ━━━━━━━━━━━━━━━━━━━━ 11s 27ms/step - accuracy: 0.9997 - loss: 0.0020 - val_accuracy: 0.9414 - val_loss: 0.2834
Epoch 7/10
416/416 ━━━━━━━━━━━━━━━━━━━━ 13s 30ms/step - accuracy: 1.0000 - loss: 9.2378e-04 - val_accuracy: 0.9405 - val_loss: 0.3072
Epoch 8/10
416/416 ━━━━━━━━━━━━━━━━━━━━ 12s 29ms/step - accuracy: 0.9969 - loss: 0.0087 - val_accurac

2024-11-28 15:42:59.880436: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 83200000 exceeds 10% of free system memory.


130/130 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step
Accuracy: 0.5620192307692308
              precision    recall  f1-score   support

           0       0.56      0.57      0.56      2077
           1       0.56      0.56      0.56      2083

    accuracy                           0.56      4160
   macro avg       0.56      0.56      0.56      4160
weighted avg       0.56      0.56      0.56      4160



# BiLSTM Model with Word2Vec

In [ ]:
import keras
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.layers import Dense, Dropout, BatchNormalization
from keras.models import Sequential
from sklearn.metrics import accuracy_score, classification_report

# Source https://www.tensorflow.org/text/tutorials/text_classification_rnn

preprocessor = TextPreprocessor()

train_df = pd.read_csv('train.csv')
train_texts = train_df['text'].apply(preprocessor.clean_text)
train_labels = train_df['label']


X_train, X_test, y_train, y_test = train_test_split(
    train_texts, train_labels, test_size=0.2, random_state=42, stratify=train_labels
)

X_train_text_vectors = []
X_test_text_vectors = []

word2vec_model = preprocessor.get_word2vec_features(X_train)

for text in X_train:
    text_vector = preprocessor.get_text_vector(text, word2vec_model)
    X_train_text_vectors.append(text_vector)
    
for text in X_test:
    text_vector = preprocessor.get_text_vector(text, word2vec_model)
    X_test_text_vectors.append(text_vector)

X_train_text_vectors = pd.DataFrame(X_train_text_vectors)
X_test_text_vectors = pd.DataFrame(X_test_text_vectors)

X_train_word2Vec = X_train_text_vectors.to_numpy()
X_test_word2Vec = X_test_text_vectors.to_numpy()




num_classes = len(set(train_labels))
y_train_encoded = keras.utils.to_categorical(y_train, num_classes)
y_test_encoded = keras.utils.to_categorical(y_test, num_classes)



model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train_word2Vec.shape[1],)))
model.add(BatchNormalization())  # Add batch normalization
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))


model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


model.fit(
    X_train_word2Vec,
    y_train_encoded,
    batch_size=32,
    epochs=50,
    validation_split=0.2
)


y_pred_prob = model.predict(X_test_word2Vec) 
y_pred = np.argmax(y_pred_prob, axis=1)   
y_test_labels = np.argmax(y_test_encoded, axis=1) 


print("Accuracy:", accuracy_score(y_test_labels, y_pred))
print(classification_report(y_test_labels, y_pred))


# BiLSTM Model

In [ ]:
import keras
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Dropout, Embedding, Bidirectional, LSTM
from keras.models import Sequential
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences

# Source https://www.tensorflow.org/text/tutorials/text_classification_rnn

preprocessor = TextPreprocessor()
tokenizer = Tokenizer()
max_sequence_length = 200

train_df = pd.read_csv('train.csv')
train_texts = train_df['text'].apply(preprocessor.clean_text)
train_labels = train_df['label']


X_train, X_test, y_train, y_test = train_test_split(train_texts, train_labels, test_size=0.2, random_state=42, stratify=train_labels)

tokenizer.fit_on_texts(X_train)
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_train_sequences = pad_sequences(X_train_sequences, maxlen=max_sequence_length)

X_test_sequences = tokenizer.texts_to_sequences(X_test)
X_test_sequences = pad_sequences(X_test_sequences, maxlen=max_sequence_length)


X_train_sequences = pd.DataFrame(X_train_sequences)
X_test_sequences = pd.DataFrame(X_test_sequences)

num_classes = len(set(train_labels))
y_train_encoded = keras.utils.to_categorical(y_train, num_classes)
y_test_encoded = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=max_sequence_length))
model.add(Bidirectional(LSTM(64, return_sequences=False)))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))  
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])


model.fit(
    X_train_sequences,
    y_train_encoded,
    batch_size=32,
    epochs=3,
    validation_split=0.2
)


y_pred_prob = model.predict(X_test_sequences)  
y_pred = np.argmax(y_pred_prob, axis=1)   
y_test_labels = np.argmax(y_test_encoded, axis=1) 

print("Accuracy:",  accuracy_score(y_test_labels, y_pred))
print(classification_report(y_test_labels, y_pred))


Epoch 1/3


/home/kevin/.local/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


416/416 ━━━━━━━━━━━━━━━━━━━━ 200s 460ms/step - accuracy: 0.7829 - loss: 0.4229 - val_accuracy: 0.9303 - val_loss: 0.1759
Epoch 2/3
416/416 ━━━━━━━━━━━━━━━━━━━━ 170s 410ms/step - accuracy: 0.9734 - loss: 0.0802 - val_accuracy: 0.9351 - val_loss: 0.1999
Epoch 3/3
416/416 ━━━━━━━━━━━━━━━━━━━━ 160s 384ms/step - accuracy: 0.9950 - loss: 0.0182 - val_accuracy: 0.9264 - val_loss: 0.2844
130/130 ━━━━━━━━━━━━━━━━━━━━ 10s 70ms/step
Accuracy: 0.9305288461538461
              precision    recall  f1-score   support

           0       0.90      0.97      0.93      2077
           1       0.97      0.89      0.93      2083

    accuracy                           0.93      4160
   macro avg       0.93      0.93      0.93      4160
weighted avg       0.93      0.93      0.93      4160

